# Food Hunt in downtown Toronto (Part 2)

# Load Packages

In [3]:
!pip install folium

     |████████████████████████████████| 102kB 11.3MB/s ta 0:00:01


In [11]:
!pip install pgeocode

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

In [5]:
def locator(df, postal_code, country_code):
    import pgeocode
    import pandas

    nomi = pgeocode.Nominatim(country_code)
    df['lat'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).latitude)
    df['lng'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).longitude)

    return df

In [7]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')
wikitable = soup.find('table')
columns = [i.text.strip() for i in wikitable.find_all('th')]
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]
trtdata = pd.DataFrame(data, columns = columns)
trtdata['Neighborhood'] = trtdata.Neighborhood.apply(lambda x: x.replace(' /',','))
trtdata.columns = ['PostalCode','Borough', 'Neighborhood']

In [12]:
trtdata1 = trtdata.loc[trtdata.Borough != 'Not assigned'].reset_index(drop=True)
trtdata1.loc[trtdata1.Neighborhood == 'Not assigned', 'Neighborhood'] = trtdata['Borough']
trtdata2 = trtdata1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()
trtdata3 = locator(trtdata2, 'PostalCode', 'ca')
trtdata3 = trtdata3.dropna()

In [13]:
trtdata3.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [8]:
# The code was removed by Watson Studio for sharing.

In [21]:
radius=1000
LIMIT=10
VERSION = '20200605'
lat = trtdata3.lat[0]
lng = trtdata3.lng[0]

In [22]:

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

# make the GET request
results = requests.get(url).json()

In [23]:
results

{'meta': {'code': 200, 'requestId': '5f0bcd05a4255c38437d1035'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.820300009000015,
    'lng': -79.18055139991516},
   'sw': {'lat': 43.80229999099999, 'lng': -79.20544860008484}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c97975582b56dcb8320ebaa',
       'name': 'Canadiana exhibit',
       'location': {'lat': 43.81796218928876,
        'lng': -79.19337359666939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81796218928876,
          'lng': -79.19337359666939}],
        'distance': 742,
        'cc': 'CA',
        'neighborhood': 'Rouge',
        'city': 'Toronto',
    